# Selektory akcji

In [1]:
import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)

In [2]:
import ptan
import numpy as np
import torch
import torch.nn as nn

/usr/local/lib/python3.11/dist-packages/ignite/handlers/checkpoint.py:16: DeprecationWarning: `TorchScript` support for functional optimizers is deprecated and will be removed in a future PyTorch release. Consider using the `torch.compile` optimizer instead.
  from torch.distributed.optim import ZeroRedundancyOptimizer


In [3]:
q_vals = np.array([[1, 2, 3], [1, -1, 0]])
q_vals

/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


array([[ 1,  2,  3],
       [ 1, -1,  0]])

In [4]:
selector = ptan.actions.ArgmaxActionSelector()
print("argmax:", selector(q_vals))

argmax: [2 0]


In [5]:
selector = ptan.actions.EpsilonGreedyActionSelector(epsilon=0.0)
print("epsilon=0.0:", selector(q_vals))

epsilon=0.0: [2 0]


In [6]:
selector.epsilon = 0.5
print("epsilon=0.5:", selector(q_vals))
selector.epsilon = 0.1
print("epsilon=0.1:", selector(q_vals))

epsilon=0.5: [2 1]
epsilon=0.1: [2 0]


In [7]:
selector = ptan.actions.ProbabilityActionSelector()
print("Akcje wybrane z trzech rozkładów prawdopodobieństwa:")
for _ in range(10):
    acts = selector(np.array([
          [0.1, 0.8, 0.1],
          [0.0, 0.0, 1.0],
          [0.5, 0.5, 0.0]
        ]))
    print(acts)

Akcje wybrane z trzech rozkładów prawdopodobieństwa:
[0 2 0]
[1 2 1]
[0 2 1]
[1 2 0]
[1 2 0]
[1 2 0]
[1 2 0]
[1 2 0]
[1 2 1]
[0 2 0]


# Agent

In [8]:
# DQNAgent
# stosowany, gdy przestrzeń akcji nie jest zbyt duża (np. gry Atari)
# dane wejściowe - paczka obserwacji w formie tablicy NumPy
# obserwacje przekazywane są do sieci w celu uzyskania wartości Q
# następnie stosowany jest ActionSelector do konwersji tych wartości na indeksy odpowiadające akcjom

class DQNNet(nn.Module):
    def __init__(self, actions: int):
        super(DQNNet, self).__init__()
        self.actions = actions

    def forward(self, x):
		  # zawsze tworzymy diagonalny tensor o kształcie (rozmiar_paczki, akcje)
        return torch.eye(x.size()[0], self.actions)

In [9]:
net = DQNNet(actions=3) # używamy agenta jako modelu głębokiej sieci Q
net_out = net(torch.zeros(2, 10))
print("dqn_net:")
print(net_out)

dqn_net:
tensor([[1., 0., 0.],
        [0., 1., 0.]])


In [10]:
selector = ptan.actions.ArgmaxActionSelector() # polityka argmax - agent zwróci akcje odpowiadające wartościom 1.0 w danych wyjściowych sieci
agent = ptan.agent.DQNAgent(model=net, action_selector=selector)
ag_out = agent(torch.zeros(2, 5)) # dane wejściowe - dwie obserwacje po pięć wartości
print("Argmax:", ag_out) # agent zwraca krotkę z dwoma obiektami:
# 1. tablica z akcjami do wykonania dla obu obserwacji
# 2. lista ze stanem wewnętrznym agenta (obecny agent bezstanowy, lista zawiera wartości None)

Argmax: (array([0, 1]), [None, None])


In [11]:
selector = ptan.actions.EpsilonGreedyActionSelector(epsilon=1.0) # polityka epsilonu zachłannego
#wszystkie akcje będą losowe, niezależnie od danych wyjściowych sieci
agent = ptan.agent.DQNAgent(model=net, action_selector=selector)
ag_out = agent(torch.zeros(10, 5))[0]
print("eps=1.0:", ag_out)

selector.epsilon = 0.5 # możemy "w locie" zmniejszać tę wartość
# jest to przydatne podczas treningu i stopniowego zmniejszania epsilonu
ag_out = agent(torch.zeros(10, 5))[0]
print("eps=0.5:", ag_out)

selector.epsilon = 0.1
ag_out = agent(torch.zeros(10, 5))[0]
print("eps=0.1:", ag_out)

eps=1.0: [0 0 0 2 2 2 0 1 0 0]
eps=0.5: [0 1 2 0 0 0 2 2 0 0]
eps=0.1: [0 1 2 0 0 1 0 0 0 0]


In [12]:
# PolicyNet
# obiekt tej klasy oczekuje generowania przez sieć polityki w postaci dyskretnego zestawu akcji
# dystrybucja ta może być nieznormalizowana (realizowana funkcją logitową) lub znormalizowana

class PolicyNet(nn.Module):
    def __init__(self, actions: int):
        super(PolicyNet, self).__init__()
        self.actions = actions

    def forward(self, x):
		  # Teraz tworzymy tensor z dwiema pierwszymi akcjami
		  # o tych samych wartościach funkcji logitowej
        shape = (x.size()[0], self.actions)
        res = torch.zeros(shape, dtype=torch.float32)
        res[:, 0] = 1
        res[:, 1] = 1
        return res # sieć generuje wartości prawdopodobieństw

In [13]:
net = PolicyNet(actions=5)
net_out = net(torch.zeros(6, 10))
print("policy_net:")
print(net_out)

policy_net:
tensor([[1., 1., 0., 0., 0.],
        [1., 1., 0., 0., 0.],
        [1., 1., 0., 0., 0.],
        [1., 1., 0., 0., 0.],
        [1., 1., 0., 0., 0.],
        [1., 1., 0., 0., 0.]])


In [14]:
selector = ptan.actions.ProbabilityActionSelector()
agent = ptan.agent.PolicyAgent(model=net, action_selector=selector, apply_softmax=True)
# klasa PolicyAgent powinna zastosować do danych wyjściowych sieci funkcję softmax
ag_out = agent(torch.zeros(6, 5))[0]
print(ag_out)

[0 1 1 1 1 1]


In [15]:
nn.functional.softmax(net(torch.zeros(1, 10)), dim=1)
# funkcja softmax zwróci niezerowe prawdopodobieństwa, również dla logitów = 0
# agent może więc wybrać więcej niż jedną akcję

tensor([[0.3222, 0.3222, 0.1185, 0.1185, 0.1185]])